In [1]:
import sqlite3

In [2]:
#creating a new sqlite database or connect to an existing database
"""
   If the specified database does not exist,the sqlite3.connect function 
   will create a new one
"""
conn = sqlite3.connect("my_database.db")

In [3]:
#create a cursor object which allows you to execute commands
cursor = conn.cursor()

In [4]:
#CREATE TABLE statement is used to define the structure of a table including it name,columns and data types
#create a table called users with columns id,name and email
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  email TEXT NOT NULL UNIQUE
)
""")

In [5]:
#To insert data  into tables ,we use the INSERT INTO statement
#insert new user the users table
cursor.execute("""
INSERT INTO users (name,email)
VALUES (?,?)
""",("John Doe","johndoe@example.com"))

In [6]:
#commit the transacation to save changes in the database
conn.commit()

In [7]:
#To query data from a table we use the SELECT statement
# -> select all users from the users table
cursor.execute("SELECT * FROM users")

#Fetch all rows from the result set
rows = cursor.fetchall() #returns a list of tuples
for row in rows:
    print(row)

(1, 'John Doe', 'johndoe@example.com')


In [8]:
#To update data in a table we use the UPDATE statement
# -> update the email and adress of a user
cursor.execute("""
UPDATE users
SET email = ?
WHERE name = ?
""",("new_email@example.com","John Doe"))

#commit the transcation
conn.commit()

In [9]:
#To delete data from a table we use DELETE FROM statement
# -> Delete a user from the users tables
cursor.execute("""
DELETE FROM users
WHERE name = ?
""",("John Doe",))

#commit the transaction
conn.commit()

In [10]:
#starting a new data transcato use conn.begin()

In [13]:
#handiling errors
"""
   try:
     #perform a databse operation
     cursor.execute("INSERT INTO users (name,email) VALUES (?,?)",("Jane Doe","Jane_doe@example.com"))
     conn.commit()
    except sqlite3.Error as e:
      print(f'An error occured: {e}')
      conn.rollback()
"""

'\n   try:\n     #perform a databse operation\n     cursor.execute("INSERT INTO users (name,email) VALUES (?,?)",("Jane Doe","Jane_doe@example.com"))\n     conn.commit()\n    except sqlite3.Error as e:\n      print(f\'An error occured: {e}\')\n      conn.rollback()\n'

In [14]:
#Indexes can be used to speed up the process of searching and sorting data in a table
#To create an index you can use CREATE INDEX statement
cursor.execute("CREATE INDEX IF NOT EXISTS users_email_index ON users (email)")

In [15]:
#Views are virtual tables that are based on the result of SELECT statement
# -> create a new view thet return the user's name and email adress
cursor.execute("""
CREATE VIEW IF NOT EXISTS user_info AS
SELECT name,email
FROM users
""")

In [16]:
#View can be queried like regular tables 
cursor.execute("SELECT * FROM user_info")

#Fetch all rows from the result set
rows = cursor.fetchall()
#print the rows
for row in rows:
    print(row)

In [18]:
#Triggers are database objects that are automatically executed when an event such as an INSERT,UPDATE or DELETE occurs on a 
#specified tables.
#This can be used to perform actions such a enforcing data integrity,maintaining a history of changes or updating related tables
# -> create a trigger that logs changes to the user tables
cursor.execute("""
CREATE TRIGGER IF NOT EXISTS users_log_changes
AFTER UPDATE ON users
BEGIN
    INSERT INTO users_log(user_id,old_email,new_email)
    VALUES (old.id,old.email,new.email);
END;
""")

In [19]:
#SQLite supports full-text search using the FTS3,FTS4 and FTS5 extensions
#To use full-text search in SQLite , you need to create a virtual tables using the FTS extension and define the columns indexed
cursor.execute("""
CREATE VIRTUAL TABLE IF NOT EXISTS users_fts USING fts5(name,email);
""")

In [20]:
#Use the MATCH operator to search for text in the index column
cursor.execute("""
SELECT * FROM users_fts WHERE users_fts MATCH 'John Doe'
""")